In [13]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session



In [14]:
#read excel
fake_df = pd.read_excel('example_datatable.xlsx')
df = pd.read_excel('example_datatable.xlsx')

In [20]:
fake_df

,sample,age_gap,edu_gap,children,race_gap,homosexual
0,1,2,y,1.0,n,n
1,2,23,y,2.0,n,n
2,3,64,y,2.0,y,n
3,4,2,y,3.0,y,n
4,5,46,n,0.0,y,n
5,6,22,n,0.0,y,n
6,7,3,y,0.0,n,n
7,11,5,n,0.0,n,y
8,22,8,n,0.0,y,n
9,33,5,n,1.0,n,n


In [16]:
database_path = "fake_df_2"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [17]:
fake_df('fake_df', metadata,
        Column('id', Integer, primary_key=True),
        sqlite_autoincrement=True)

NameError: name 'metadata' is not defined

In [22]:
connection_string = "fake_data.sqlite"
engine2 = create_engine(f'sqlite:///{connection_string}')
engine2.table_names() # now it's empty

['fake_df', 'sample_metadata', 'wv2_w_2', 'x']

In [23]:
fake_df.to_sql(name="fake_df", con=engine2, if_exists='replace', index=True)
engine2.table_names()

['fake_df', 'sample_metadata', 'wv2_w_2', 'x']

ImportError: cannot import name 'curser' from 'sqlite3' (C:\Users\Romina\Anaconda3\lib\sqlite3\__init__.py)

In [30]:
# CREATE UNIQUE INDEX mytable_fake_pk ON mytable(pk_column)
#connect to the database
conn = sqlite3.connect('database')
c = conn.curser()
c.executescript('''CREATE UNIQUE INDEX mytable_fake_pk ON fake_df(sample)''')

AttributeError: 'sqlite3.Connection' object has no attribute 'curser'

In [26]:
import sqlite3

# df = pd.read_csv("/Users/data/" +filename) 
# columns = df.columns columns = [i.replace(' ', '_') for i in columns]
columns = fake_df.columns 
columns = [i.replace(' ', '_') for i in columns]

#write the pandas dataframe to a sqlite table
fake_df.columns = columns
fake_df.to_sql(name,con,flavor='sqlite',schema=None,if_exists='replace',index=True,index_label=None, chunksize=None, dtype=None)

#connect to the database
conn = sqlite3.connect('database')
c = conn.curser()

c.executescript('''
    PRAGMA foreign_keys=off;
# edited to fake_df
    BEGIN TRANSACTION;
    ALTER TABLE table fake_df TO old_table;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE new_table (col_1 TEXT PRIMARY KEY NOT NULL,
                            col_2 TEXT);

    INSERT INTO new_table SELECT * FROM old_table;

    DROP TABLE old_table;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()

NameError: name 'name' is not defined

In [33]:
import re

def get_create_table_string(tablename, connection):
    sql = """
    select * from sqlite_master where name = "{}" and type = "table"
    """.format(tablename) 
    result = connection.execute(sql)

    create_table_string = result.fetchmany()[0][4]
    return create_table_string

def add_pk_to_create_table_string(create_table_string, colname):
    regex = "(\n.+{}[^,]+)(,)".format(colname)
    return re.sub(regex, "\\1 PRIMARY KEY,",  create_table_string, count=1)

def add_pk_to_sqlite_table(tablename, index_column, connection):
    cts = get_create_table_string(tablename, connection)
    cts = add_pk_to_create_table_string(cts, index_column)
    template = """
    BEGIN TRANSACTION;
        ALTER TABLE {tablename} RENAME TO {tablename}_old_;

        {cts};

        INSERT INTO {tablename} SELECT * FROM {tablename}_old_;

        DROP TABLE {tablename}_old_;

    COMMIT TRANSACTION;
    """

    create_and_drop_sql = template.format(tablename = tablename, cts = cts)
    connection.executescript(create_and_drop_sql)

In [27]:
def add_pk_to_sqlite_table(tablename, index_column, connection):
    cts = get_create_table_string(tablename, connection)
    cts = add_pk_to_create_table_string(cts, index_column)
    template = """
    BEGIN TRANSACTION;
        ALTER TABLE {tablename} RENAME TO {tablename}_old_;

        {cts};

        INSERT INTO {tablename} SELECT * FROM {tablename}_old_;

        DROP TABLE {tablename}_old_;

    COMMIT TRANSACTION;
    """

    create_and_drop_sql = template.format(tablename = tablename, cts = cts)
    connection.executescript(create_and_drop_sql)

In [34]:

# df = pd.DataFrame({"a": [1,2,3], "b": [2,3,4]})
con = sqlite3.connect("deleteme.db")
df.to_sql("df", con, if_exists="replace")

add_pk_to_sqlite_table("df", "index", con)
r = con.execute("select sql from sqlite_master where name = 'df' and type = 'table'")
print(r.fetchone()[0])

CREATE TABLE "df" (
"index" INTEGER PRIMARY KEY,
  "sample" INTEGER,
  "age_gap" INTEGER,
  "edu_gap" TEXT,
  "children" REAL,
  "race_gap" TEXT,
  "homosexual" TEXT
)


In [ ]:
# class SomeClass(Base):
class SomeClass(engine2):
    __tablename__ = "sample_metadata"

    uid = sample(Integer, primary_key=True)
#     bar = Column(String, primary_key=True)

In [ ]:
sqlite_wv2_w=engine2.execute("SELECT * FROM wv2_w_2").fetchall()
for elem in sqlite_wv2_w[0:5]:
    print(elem)
    print("\n")

In [ ]:
# import sqlite3
# import pandas as pd

# # Create the connection
# cnx = sqlite3.connect(r'C:\mydatabases\bigdata.db')

# # create the dataframe from a query
# df = pd.read_sql_query("SELECT * FROM userdata", cnx)